In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'e:\\End_to_end_Wine_Quality_Prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [16]:
from Wine_quality_prediction.constants import *
from Wine_quality_prediction.utils.common import read_yaml,create_directories
from Wine_quality_prediction import logger

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

      

        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:

        config = self.config.model_trainer
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        

        model_trainer_config= ModelTrainerConfig(
            root_dir=config.root_dir,
            train_path=config.data.train_path,
            test_path=config.data.test_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )

        return model_trainer_config


In [8]:
import pandas as pd
import os
from Wine_quality_prediction import logger
from sklearn.linear_model import ElasticNet
import joblib

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    

    def train(self):

        train_data=pd.read_csv(self.config.train_path)
        test_data=pd.read_csv(self.config.test_path)

        train_x=train_data.drop([self.config.target_column],axis=1)
        train_y=train_data([self.config.target_column])
        test_x=test_data.drop([self.config.target_column],axis=1)
        test_y=test_data([self.config.target_column])


        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio)
        lr.fit(train_x, train_y)


        joblib.dump(lr, os.path.join(self.config.root_dir,self.config.model_name))

In [19]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    logger.info("Creating object for Model trainer config")
    model_trainer= ModelTrainer(config=model_trainer_config)
    model_trainer.train()
    
except Exception as e:
    raise e

[2024-08-02 00:02:16,430: INFO: common: yaml_file:config\config.yaml loaded successfully]
[2024-08-02 00:02:16,433: INFO: common: yaml_file:params.yaml loaded successfully]
[2024-08-02 00:02:16,438: INFO: common: yaml_file:schema.yaml loaded successfully]
[2024-08-02 00:02:16,441: INFO: common: Created directory at artifacts]
[2024-08-02 00:02:16,442: INFO: common: Created directory at artifacts/model_trainer]


BoxKeyError: "'ConfigBox' object has no attribute 'artifacts_root'"